In [1]:
#!conda install openpyxl

In [2]:
# import pandas module
import pandas as pd
import re
import numpy as np

# creating a data frame
data = pd.read_excel('Site_Cherfech.xlsx')#, usecols=['Column17', 'Column18', 'Column25', 'Column26'])
data.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,Column1,Column2,Column9,Column10,Column17,Column18,Column25,Column26
1,NaN,A1_4xAA,Soil_tension / SoilHumiditySensor,A2_solar_x2_01,Soil_moisture / SoilHumiditySensor,A3_solar_x2_02,Soil_tension / SoilHumiditySensor,A4_solar_x1_01,Soil_tension / SoilHumiditySensor
2,NaN,2022-12-08T13:55:07+01:00,0,2022-12-08T13:51:08+01:00,0,2022-12-08T13:57:27+01:00,0,2022-12-08T13:50:41+01:00,0
3,NaN,2022-12-08T14:04:18+01:00,0,2022-12-08T14:00:33+01:00,0,2022-12-08T14:07:00+01:00,0,2022-12-08T14:00:22+01:00,0
4,NaN,2022-12-08T14:04:18+01:00,0,2022-12-08T14:00:33+01:00,0,2022-12-08T14:07:00+01:00,0,2022-12-08T14:00:22+01:00,0


In [3]:
df_a3 = data[['Unnamed: 5', 'Unnamed: 6']]
df_a3 = df_a3[2:]
df_a3 = df_a3.rename(columns={'Unnamed: 5': 'Time1'})
df_a3 = df_a3.rename(columns={'Unnamed: 6': 'A3_soil_moisture_in_cb'})
df_a3['Time1']=pd.to_datetime(df_a3['Time1'])
df_a3 = df_a3.set_index('Time1')
df_a3.tail()

,A3_soil_moisture_in_cb
Time1,
2023-06-03 02:37:37+01:00,0
2023-06-03 02:47:13+01:00,0
2023-06-05 12:14:55+01:00,0
2023-06-05 12:24:30+01:00,0
2023-06-05 12:34:10+01:00,0


In [4]:
df_a4 = data[['Unnamed: 7', 'Unnamed: 8']]
df_a4 = df_a4[2:]
df_a4 = df_a4.rename(columns={'Unnamed: 7': 'Time2'})
df_a4 = df_a4.rename(columns={'Unnamed: 8': 'A4_soil_moisture_in_cb'})
df_a4['Time2']=pd.to_datetime(df_a4['Time2'])
df_a4 = df_a4.set_index('Time2')
df_a4.tail()

,A4_soil_moisture_in_cb
Time2,
NaT,NaN
NaT,NaN
NaT,NaN
NaT,NaN
NaT,NaN


In [5]:
# Remove rows with missing values in df2
df_a4.dropna(inplace=True)

In [6]:
type(df_a4)

pandas.core.frame.DataFrame

In [7]:
# Merge using merge_asof
merged_df = pd.merge_asof(df_a3, df_a4, left_on='Time1', right_index=True, direction='backward')

#merged_df = pd.merge_asof(df_a3, df_a4, left_on='Time1', right_on='Time2', direction='backward')

In [8]:
merged_df.rename(columns={'Time1': 'Time'})

,A3_soil_moisture_in_cb,A4_soil_moisture_in_cb
Time1,,
2022-12-08 13:57:27+01:00,0,0
2022-12-08 14:07:00+01:00,0,0
2022-12-08 14:07:00+01:00,0,0
2022-12-08 14:16:33+01:00,0,0
2022-12-08 14:26:07+01:00,0,0
...,...,...
2023-06-03 02:37:37+01:00,0,0
2023-06-03 02:47:13+01:00,0,0
2023-06-05 12:14:55+01:00,0,0


In [9]:
merged_df.head()

,A3_soil_moisture_in_cb,A4_soil_moisture_in_cb
Time1,,
2022-12-08 13:57:27+01:00,0,0
2022-12-08 14:07:00+01:00,0,0
2022-12-08 14:07:00+01:00,0,0
2022-12-08 14:16:33+01:00,0,0
2022-12-08 14:26:07+01:00,0,0


In [10]:
merged_df[['A3_soil_moisture_in_cb', 'A4_soil_moisture_in_cb']] = merged_df[['A3_soil_moisture_in_cb', 'A4_soil_moisture_in_cb']].apply(pd.to_numeric, errors='coerce')


In [11]:
merged_df.dtypes

A3_soil_moisture_in_cb    float64
A4_soil_moisture_in_cb    float64
dtype: object

In [12]:
data_re = merged_df.resample('60T').mean().round(2)
data_re.head()

,A3_soil_moisture_in_cb,A4_soil_moisture_in_cb
Time1,,
2022-12-08 13:00:00+01:00,0.0,0.0
2022-12-08 14:00:00+01:00,0.0,0.0
2022-12-08 15:00:00+01:00,0.0,0.0
2022-12-08 16:00:00+01:00,0.0,0.0
2022-12-08 17:00:00+01:00,0.0,0.0


In [13]:
data_re = data_re.iloc[:4238]

In [14]:
data_re.to_csv("site_1_data_resampled_only_pressure.csv", index=True) 